In [1]:
import pandas as pd
import glob
import os
import ner_functions as nf

In [2]:
# 定义一个字典储存每个国家的别名
country_aliases = {
"United States": ["USA", "America", "US", "United States"],
"Canada": ["Canada"],
"United Kingdom": ["UK", "United Kingdom", "Britain", "England", "Scotland", "Wales", "Northern Ireland"],
"Australia": ["Australia", "AU", "Aussie"],
"China": ["China", "PRC"],
"Denmark": ["Denmark", "DK"],
"Finland": ["Finland", "FI"],
"France": ["France", "French Republic", "FR"],
"Germany": ["Germany", "DE"],
"Japan": ["Japan", "JP"],
"Italy": ["Italy", "Italian Republic", "IT"],
"Netherlands": ["Netherlands", "Holland", "NL"],
"Norway": ["Norway", "NO"],
"Portugal": ["Portugal", "PT"],
"Singapore": ["Singapore", "SG"],
"South Korea": ["South Korea", "KR"],
"Spain": ["Spain", "ES"],
"Sweden": ["Sweden", "SE"],
"Switzerland": ["Switzerland", "Swiss Confederation", "Swiss", "CH"],
"New Zealand": ["New Zealand", "NZ"]
}

In [3]:
lst_files = []

Path = "../Data/Articles/ReutersArticles/*.csv"

for fname in glob.glob(Path):
    lst_files.append(fname)

lst_files = sorted(lst_files)

# NER判定的阈值
threshold = 0

for file in lst_files:
    file_name = os.path.basename(file)  # Get the file name from the full file path
    country_name = os.path.splitext(file_name)[0]  # Remove the file extension
    country_name = country_name.replace("_articles", "")  # Remove the "_articles" part of the file name

    # 仅对单个国家的数据进行实体识别
    # if country_name == "Canada":
    # 对所有的未进行过NER的国家进行实体识别
    if os.path.isfile('../Data/NER/count_3/Reuters_NER/{0}.csv'.format(country_name)):
        print("File already exists: {0}.csv".format(country_name))
    else:
        print(file)
        df = pd.read_csv(file)

        if df.shape[0] != 0:
            
            df = nf.preprocess_dataframe(df)

            df, df_ner_results = nf.perform_ner_on_dataframe(df, country_name, country_aliases, threshold)


            # Convert the list of relevant articles back to a DataFrame
            df.to_csv(r'../Data/NER/count_3/Reuters_NER/{0}.csv'.format(country_name))
            df_ner_results.to_csv(r'../Data/NER/count_3/Reuters_NER_Results/{0}.csv'.format(country_name))

../Data/Articles/ReutersArticles\Australia_articles.csv


Processing Australia:   0%|          | 0/11187 [00:00<?, ?it/s]

../Data/Articles/ReutersArticles\Canada_articles.csv


Processing Canada:   0%|          | 0/8908 [00:00<?, ?it/s]

../Data/Articles/ReutersArticles\China_articles.csv


...

Processing Switzerland:   0%|          | 0/2726 [00:00<?, ?it/s]

../Data/Articles/ReutersArticles\United Kingdom_articles.csv


Processing United Kingdom:   0%|          | 0/14940 [00:00<?, ?it/s]

../Data/Articles/ReutersArticles\United States_articles.csv


Processing United States:   0%|          | 0/22907 [00:00<?, ?it/s]

In [4]:
# 读取所有国家的实体识别后剩余的数据, 然后输出剩余数据的数量
lst_ner_files = []
for fname in glob.glob("../Data/NER/count_3/Reuters_NER/*.csv"):
    lst_ner_files.append(fname)
for file in lst_ner_files:
    file_name = os.path.basename(file)  # Get the file name from the full file path
    country_name = os.path.splitext(file_name)[0]  # Remove the file extension
    df = pd.read_csv(file)
    print(country_name, df.shape[0])

Australia 1744
Canada 1670
China 2574
Denmark 7
Finland 30
France 558
Germany 400
Italy 400
Japan 1902
Netherlands 66
New Zealand 548
Norway 43
Portugal 59
Singapore 490
South Korea 7
Spain 122
Sweden 36
Switzerland 169
United Kingdom 2891
United States 1326
